# 测试专用文件

## 正常流程

### 1.Prepare for inference

In [ ]:
import sys
sys.path.append('/data3/fanpeishan/state/for_state/scripts')
from prepare_test_data import prepare_test_data
prepare_test_data(
    data_in="/data1/fanpeishan/State-Tahoe-Filtered/c37.h5ad",
    data_out="/data3/fanpeishan/state/for_state/run_results/run14/c37_prep.h5ad",
    var_dims_path="/data3/fanpeishan/state/for_state/models/ST-Tahoe/var_dims.pkl"
)

### 2. preprocess infer

In [ ]:
export CUDA_VISIBLE_DEVICES=2
state tx preprocess_infer \
  --adata /data3/fanpeishan/state/for_state/run_results/run14/c37_prep.h5ad \
  --output /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess.h5ad \
  --pert-col drugname_drugconc \
  --control-condition "[('DMSO_TF', 0.0, 'uM')]"

In [ ]:
Set random seed to 42
Identifying control cells with condition: "[('DMSO_TF', 0.0, 'uM')]"
Found 45150 control cells out of 1835947 total cells
Processing 1136 non-control perturbations
Replacement complete! Replaced expression in 1790797 cells
Control cells (45150) retain their original expression
============================================================
PREPROCESSING SUMMARY:
  - Input: 1835947 cells, 2000 genes
  - Control condition: "[('DMSO_TF', 0.0, 'uM')]"
  - Control cells: 45150 (unchanged)
  - Perturbed cells: 1790797 (replaced with control expression)
  - Perturbations processed: 1136
  - Using expression matrix (X)

USAGE:
  The output file contains cells with control expression but original
  perturbation annotations. When passed through state_transition inference,
  the model will apply perturbation effects to simulate the original data.
  Compare: state_transition(output) ≈ original_input
============================================================
Saving preprocessed data to /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess.h5ad
Preprocessing complete!

### 3. infer


In [ ]:
export CUDA_VISIBLE_DEVICES=1
state tx infer \
    --model-dir /data3/fanpeishan/state/for_state/models/ST-Tahoe \
    --checkpoint /data3/fanpeishan/state/for_state/models/ST-Tahoe/final_from_preprint.ckpt \
    --pert-col 'drugname_drugconc' \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --celltype-col cell_name \
    --batch-col 'plate' \
    --adata /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess.h5ad \
    --output /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess_infer.h5ad \
    --quiet

In [ ]:
StateTransitionPerturbationModel(
  (loss_fn): SamplesLoss()
  (pert_encoder): Sequential(
    (0): Linear(in_features=1138, out_features=1488, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1488, out_features=1488, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1488, out_features=1488, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=1488, out_features=1488, bias=True)
  )
  (basal_encoder): Sequential(
    (0): Linear(in_features=2000, out_features=1488, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1488, out_features=1488, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1488, out_features=1488, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=1488, out_features=1488, bias=True)
  )
  (transformer_backbone): LlamaModel(
    (embed_tokens): Embedding(32000, 1488, padding_idx=0)
    (layers): ModuleList(
      (0-5): 6 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=1488, out_features=1488, bias=False)
          (k_proj): Linear(in_features=1488, out_features=1488, bias=False)
          (v_proj): Linear(in_features=1488, out_features=1488, bias=False)
          (o_proj): Linear(in_features=1488, out_features=1488, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=1488, out_features=5952, bias=False)
          (up_proj): Linear(in_features=1488, out_features=5952, bias=False)
          (down_proj): Linear(in_features=5952, out_features=1488, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((1488,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((1488,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((1488,), eps=1e-06)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (project_out): Sequential(
    (0): Linear(in_features=1488, out_features=1488, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1488, out_features=1488, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1488, out_features=1488, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=1488, out_features=2000, bias=True)
  )
  (relu): ReLU()
)
=== Inference complete ===
Input cells:         1835947
Controls simulated:  45150
Treated simulated:   1790797
Wrote predictions to adata.X
Saved:               /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess_infer.h5ad

### 4.eval

In [ ]:
export CUDA_VISIBLE_DEVICES=1
cell-eval run \
    -ap /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess_infer.h5ad \
    -ar /data3/fanpeishan/state/for_state/run_results/run14/c37_prep.h5ad \
    -o /data3/fanpeishan/state/for_state/run_results/run14/run_results \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --pert-col 'drugname_drugconc' \
    --profile full \
    --batch-size 1024 \
    --num-threads 10 

In [6]:
import pandas as pd
results=pd.read_csv('/data3/fanpeishan/state/for_state/run_results/run14/run_results/agg_results.csv')
mean_results=results[results.statistic == 'mean'][['overlap_at_100', 'pearson_delta','mse']]
print(mean_results)

   overlap_at_100  pearson_delta       mse
2        0.080935       0.442847  0.114749


In [ ]:
import pandas as pd
results=pd.read_csv('/data3/fanpeishan/state/for_state/run_results/run14/run_results/agg_results.csv')
mean_results=results[results.statistic == 'mean'][['overlap_at_N','overlap_at_50','overlap_at_100','overlap_at_200', 'overlap_at_500']]
print(mean_results)

   overlap_at_N  overlap_at_50  overlap_at_100  overlap_at_200  overlap_at_500
2      0.220168       0.049261        0.080935        0.134311        0.206565


## test1:比较eval方法

### 1. cell-eval软件包，命令行方法

In [ ]:
export CUDA_VISIBLE_DEVICES=1
cell-eval run \
    -ap /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess_infer.h5ad \
    -ar /data3/fanpeishan/state/for_state/run_results/run14/c37_prep.h5ad \
    -o /data3/fanpeishan/state/for_state/run_results/run14/run_results \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --pert-col 'drugname_drugconc' \
    --profile full \
    --batch-size 1024 \
    --num-threads 10 

In [ ]:
import pandas as pd
results=pd.read_csv('/data3/fanpeishan/state/for_state/run_results/run14/run_results/agg_results.csv')
mean_results=results[results.statistic == 'mean'][['overlap_at_100', 'pearson_delta','mse']]
print(mean_results)

   overlap_at_100  pearson_delta       mse
2        0.080935       0.442847  0.114749


In [ ]:
import pandas as pd
results=pd.read_csv('/data3/fanpeishan/state/for_state/run_results/run14/run_results/agg_results.csv')
mean_results=results[results.statistic == 'mean'][['overlap_at_N','overlap_at_50','overlap_at_100','overlap_at_200', 'overlap_at_500']]
print(mean_results)

   overlap_at_N  overlap_at_50  overlap_at_100  overlap_at_200  overlap_at_500
2      0.220168       0.049261        0.080935        0.134311        0.206565


### 2. cell-eval软件包，python方法

In [ ]:
python /data3/fanpeishan/state/for_state/scripts/cell_eval_py.py

In [3]:
import pandas as pd
results=pd.read_csv('/data3/fanpeishan/state/for_state/run_results/run14/run_results_python/agg_eval_results.csv')
mean_results=results[results.statistic == 'mean'][['overlap_at_100', 'pearson_delta','mse']]
print(mean_results)

   overlap_at_100  pearson_delta       mse
2        0.231884       0.442847  0.114749


### 3. 学长给的eval代码

In [ ]:
python /data3/fanpeishan/state/for_state/scripts/eval_drug.py \
  --pred /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess_infer.h5ad \
  --real /data3/fanpeishan/state/for_state/run_results/run14/c37_prep.h5ad 

In [ ]:
Loading data...
[INFO] Converting real sparse matrix to dense...
[INFO] Selecting Top 2000 HVGs based on REAL data...
[WARN] Seurat HVG failed, falling back to variance ranking.
[INFO] Computing Pseudobulk...
[INFO] Running Parallel DE on REAL data (Wilcoxon, n_jobs=3)...
Real DE Analysis: 100%|███| 1136/1136 [15:44:16<00:00, 49.87s/it]
[INFO] Fast Ranking on PRED data (Simple Diff, Top 100)...
Pred Fast Ranking: 100%|███| 1136/1136 [00:00<00:00, 11512.90it/s]
Calculating metrics for 1136 perturbations...

============================================================
  FINAL EVAL_DRUG REPORT (Optimized)
  HVG: 2000 | DE Ranking: Top 100 (Abs LogFC)
============================================================
1. PCC-Delta (Accuracy)     : 0.4428
2. Common-DEGs (Recall)     : 0.0022
3. MSE (Error)              : 0.1147
============================================================